In [22]:
#Bibliotecas requeridas
import cv2
import cv2 as cv
import glob
import re
import math
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from matplotlib.colors import hsv_to_rgb
import os

def depurar_contornos(cnts):
    #en esta lista se almacenarán los contornos pertenecientes a los círculos
    #la "d" significa depurados.
    cntsd= []
    #recorro todos los contornos encontrados
    for cnt in cnts:
        area= cv2.contourArea(cnt)
        cntsd.append(area)    
    return cntsd


#Dirección de imágenes para realizar el ajuste de áreas
nombreCarpetaOut = 'Imagenes_Ajuste__Areas/'

dir = nombreCarpetaOut

#Guarda los nombres de las imágenes que estaban en la carpeta en la lista imgList
imgList = os.listdir(dir)
#Inicializa la lista que contendrá a las imágenes que se utilizarán
imagenes = []
imagenesHSV = []
#Ordena las imágenes por número de menor a mayor
imgList.sort (key = lambda x: int (x.replace ("frame", ""). split ('.') [0])) 
#Inicializa contadores
n = 0
cont = 0
ind = 0
#resolución espacial para imagen con zoom [mm/pixel]
mmPixelZ=40/54
#resolución espacial para imagen sin zoom [mm/pixel]
mmPixel=40/36

#Carga los matrices de calibración
mtx = np.loadtxt('cameraMatrix.txt', dtype=float)
newcameramtx = np.loadtxt('newCameraMatrix.txt', dtype=float)
dist = np.loadtxt('dist.txt', dtype=float)
roi = np.loadtxt('roi.txt', dtype=int)

for i in imgList:
    #va contando los frames, prevista para frames
    #cont+=1
    #Lee cada imagen, donde i es el nombre de cada una
    img_conDist = cv2.imread(nombreCarpetaOut+i)
    
    #Prevista para cada cuantos frames se quiere trabajar
    """if cont == 4:
        imagenes.append(img)     
        cont = 0
        n+=1"""
    
    #Elimina distorsión
    dst = cv.undistort(img_conDist, mtx, dist, None, newcameramtx)
    
    # recorta la imagen
    x, y, w, h = roi
    #Imagen sin distorsión
    img = dst[y:y+h, x:x+w]            
    
    #Convirtiendo a RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #Guarda las imágenes en RGB en una lista
    imagenes.append(img)
    
    #Guarda las imágenes en HSV en una lista
    imagenesHSV.append(hsv)
    #va contando las imágenes que hay
    n+=1


#--------------------- Definición de rangos de color HSV---------------------------
#Rangos cyan:
#light_cyan = (95, 175, 185)
light_cyan = (95, 175, 175)
dark_cyan = (105, 255, 250)
#Rangos magenta:
light_magenta = (145, 50, 150)
dark_magenta = (175, 255, 255)
#Rangos verde:  
light_verde = (30, 50, 50)
dark_verde = (40, 255, 255)
#Rangos rojo:
light_rojo = (168, 50, 0)
dark_rojo = (180, 255, 255)
#Rangos azul: 
#light_azul = (100, 125, 10)
light_azul = (100, 200, 100)
dark_azul = (120, 255, 255)
#Rangos naranja: 
light_nar = (9, 50, 235)
dark_nar = (20, 255, 255)
#Rangos amarillo: 
#light_ama = (15, 30, 0)
light_ama = (12, 25, 175)
dark_ama = (35, 255, 255)
#Crea un respaldo de todas las imágenes para ser modificadas más adelante
RespaldoImagenes=list(imagenes)

#------------Procesado: aplicar máscara, dilatar, contornos, ubicación -----------------
#Aquí se almacenarán los contornos de cada una de las imágenes
lista_cCyan = []
lista_cMagenta = []
lista_cVerde = []
lista_cRojo = []
lista_cAzul = []
lista_cNaranja = []
lista_cAmarillo = []
#Aquí se almacenarán los centros de los contornos
centros_cCyan = []
centros_cMagenta = []
centros_cNaranja = []
centros_cAmarillo = []
centros_cVerde = []
centros_cRojo = []
centros_cAzul = []

for j in range(len(imagenes)):
    #Máscara Cyan
    #Pone 1 en el píxel que está dentro del rango, sino pone 0
    maskcyan = cv2.inRange(imagenesHSV[j], light_cyan, dark_cyan)
    #Máscara Magenta
    maskmagenta = cv2.inRange(imagenesHSV[j], light_magenta, dark_magenta)
    #Máscara Verde
    maskverde = cv2.inRange(imagenesHSV[j], light_verde, dark_verde)
    #Máscara Amarilla
    maskama = cv2.inRange(imagenesHSV[j], light_ama, dark_ama)
    #Máscara Naranja
    masknar = cv2.inRange(imagenesHSV[j], light_nar, dark_nar)
    #Máscara Azul
    maskazul = cv2.inRange(imagenesHSV[j], light_azul, dark_azul)
    #Máscara Rojo
    maskrojo = cv2.inRange(imagenesHSV[j], light_rojo, dark_rojo)
    #Máscara de todos los colores de identificadores:
    maskAll=maskrojo+maskazul+maskama+maskcyan+maskverde+maskmagenta+masknar

    #Se utiliza una transformación morfológica de dilatación de la imagen para hacer los
    #contornos más uniformes
    #Se crea el kernel, entre mayor sea más "grosera" es la dilatación
    kernel = np.ones((7,7),np.uint8)
    cyand = cv2.dilate(maskcyan,kernel,iterations=1)
    magentad = cv2.dilate(maskmagenta,kernel,iterations=1)
    verded = cv2.dilate(maskverde,kernel,iterations=1)
    amarillod = cv2.dilate(maskama,kernel,iterations=1)
    naranjad = cv2.dilate(masknar,kernel,iterations=1)
    azuld = cv2.dilate(maskazul,kernel,iterations=1)
    rojod = cv2.dilate(maskrojo,kernel,iterations=1)
    alld = cyand+magentad+verded+amarillod+naranjad+azuld+rojod
    
    #Aplica las máscaras a las imágenes por si se quisieran visualizar
    #"""
    resultcyan = cv2.bitwise_and(imagenes[j], imagenes[j], mask=cyand)
    resultmagenta = cv2.bitwise_and(imagenes[j], imagenes[j], mask=magentad)
    resultverde = cv2.bitwise_and(imagenes[j], imagenes[j], mask=verded)
    resultama = cv2.bitwise_and(imagenes[j], imagenes[j], mask=amarillod)
    resultnar = cv2.bitwise_and(imagenes[j], imagenes[j], mask=naranjad)
    resultazul = cv2.bitwise_and(imagenes[j], imagenes[j], mask=azuld)
    resultrojo = cv2.bitwise_and(imagenes[j], imagenes[j], mask=rojod)
    #Aplica la máscara combinada
    resultAll = cv2.bitwise_and(imagenes[j], imagenes[j], mask=maskAll)
    
    '''
    #Muestra los resultados de filtrado por color para una imagen
    
    if ind==0:
        # se crea la figura
        fig = plt.figure(figsize=(8, 8))
        # definición de filas y columnas
        rows = 4
        columns = 2
        # Agrega un subplot en la primera posición
        fig.add_subplot(rows, columns, 1)
        # mostrando la imagen
        plt.imshow(resultcyan)
        plt.axis('off')
        plt.title("Cyan")
        # segunda posición
        fig.add_subplot(rows, columns, 2)
        plt.imshow(resultmagenta)
        plt.axis('off')
        plt.title("Magenta")
        # tercera posición
        fig.add_subplot(rows, columns, 3)
        plt.imshow(resultverde)
        plt.axis('off')
        plt.title("Verde")
        # cuarta posición
        fig.add_subplot(rows, columns, 4)
        plt.imshow(resultama)
        plt.axis('off')
        plt.title("Amarillo")
        # quinta posición
        fig.add_subplot(rows, columns, 5)
        plt.imshow(resultnar)
        plt.axis('off')
        plt.title("Naranja")
        # sexta posición
        fig.add_subplot(rows, columns, 6)
        plt.imshow(resultazul)
        plt.axis('off')
        plt.title("Azul")
        # séptima posición
        fig.add_subplot(rows, columns, 7)
        plt.imshow(resultrojo)
        plt.axis('off')
        plt.title("Rojo")
        # octava posición
        fig.add_subplot(rows, columns, 8)
        plt.imshow(resultAll)
        plt.axis('off')
        plt.title("Todos")
        plt.show()
    
    ind+=1
    '''
    #se inicializa la variable de tamaño de contorno
    sizeC=0
    #Busca los contornos en cada una de las imágenes filtradas
    #Se aproximan por medio de una compresión horizontal, vertical y diagonal
    #https://docs.opencv.org/master/d3/dc0/group__imgproc__shape.html#gadf1ad6a0b82947fa1fe3c3d497f260e0
    cntsCyan, jerCyan = cv2.findContours(cyand.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsMagenta, jerMagenta = cv2.findContours(magentad.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsAmarillo, jerAmarillo = cv2.findContours(amarillod.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
    cntsNaranja, jerNaranja = cv2.findContours(naranjad.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsVerde, jerVerde = cv2.findContours(verded.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cntsAzul, jerAzul = cv2.findContours(azuld.copy(),cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    #Deja solo los contornos grandes, correspondientes a los círculos pequeños y grandes
    cntsCyans=depurar_contornos(cntsCyan)
    cntsMagentas=depurar_contornos(cntsMagenta)
    cntsAmarillos=depurar_contornos(cntsAmarillo)
    cntsNaranjas=depurar_contornos(cntsNaranja)
    cntsVerde=depurar_contornos(cntsVerde)
    cntsAzul=depurar_contornos(cntsAzul)
    
    #print(cntsCyans) #Sí son detectables
    
    #print(cntsMagentas) #Sí son detectables

    #print(cntsAmarillos) #Dificilmente detectables por iluminación
    
    #print(cntsNaranjas) #Aproximadamente detectables, falta ajuste
    
    #Pruebas 15 de Junio con nuevos identificadores, colorIdentifiers(2): CYAN-verde, VERDE-cyan, MAGENTA-verde
    
    #print(cntsVerde) #Rangos ajustados buenos
    
    print(cntsCyans) 
    
    #print(cntsMagentas) 
    
#Link de ayuda:https://stackoverflow.com/questions/10948589/choosing-the-correct-upper-and-lower-hsv-boundaries-for-color-detection-withcv

[2464.0, 1301.0]
[2459.5, 1260.5]
[2476.5, 1239.0]
[2388.0, 1217.0]
[2452.5, 1264.5]
[2421.0, 1263.5]
[2485.0, 1259.0]
[2478.5, 1216.0]
[2439.5, 1215.5]
[2213.5, 31.5, 92.5, 1034.0]
[2223.5, 388.0, 1248.0]
[2297.0, 1266.0]
[2283.0, 1242.5]
